In [1]:
import pandas as pd

In [2]:
column_names = ["filename", "label", "x_center", "y_center", "width", "height"] 

In [4]:
df = pd.read_csv("corner_mapped.txt", sep=r"\s+", engine="python", names=column_names)

In [5]:
# Assume df contains the corner annotations
# and labels are: corner_tl, corner_tr, corner_br, corner_bl

# Group by image
segmentation_data = []

# Helper: map to ensure correct order
corner_order = [0, 1, 2, 3]

for filename, group in df.groupby("filename"):
    corners = {row["label"]: (row["x_center"], row["y_center"]) for _, row in group.iterrows()}

    # Only process if all 4 corners are present
    if all(corner in corners for corner in corner_order):
        ordered_coords = []
        for corner in corner_order:
            x, y = corners[corner]
            ordered_coords.extend([x, y])  # Append as x1, y1, x2, y2, ...

        segmentation_data.append({
            "filename": filename,
            "class_id": 0,  # 0 = "table_surface"
            "segmentation": ordered_coords
        })

# Create new DataFrame for YOLO segmentation format
df_seg = pd.DataFrame(segmentation_data)


In [6]:
df_seg

,filename,class_id,segmentation
0,table_0.png,0,"[0.04148895, 0.3827688, 0.7863319, 0.2552395, ..."
1,table_100.png,0,"[0.5217206, 0.9918306, 0.1662945, 0.3077753, 0..."
2,table_1000.png,0,"[0.2668231, 0.1509243, 0.8879615, 0.5201969, 0..."
3,table_1001.png,0,"[0.9462358, 0.418291, 0.3535828, 0.9011644, 0...."
4,table_1003.png,0,"[0.1673634, 0.1976223, 0.8815356, 0.4242113, 0..."
...,...,...,...
5275,table_9992.png,0,"[0.9933922, 0.7463055, 0.1201228, 0.7462301, 0..."
5276,table_9993.png,0,"[0.7773169, 0.1222933, 0.6424851, 0.875252, 0...."
5277,table_9994.png,0,"[0.8230969, 0.1761585, 0.5955555, 0.9176887, 0..."
5278,table_9998.png,0,"[0.9548575, 0.6751232, 0.1816933, 0.7552549, 0..."


In [ ]:
import os

output_folder = "corner_dataset"
os.makedirs(output_folder, exist_ok=True)

for _, row in df_seg.iterrows():
    # Create one clean string: class_id followed by space-separated coordinates
    line = f"{row['class_id']} " + " ".join([f"{coord:.6f}" for coord in row["segmentation"]])

    # Create label filename
    label_filename = row["filename"].replace(".png", ".txt")
    label_path = os.path.join(output_folder, label_filename)

    # Write to file
    with open(label_path, "w") as f:
        f.write(line + "\n")